Make sure that CUDA is available

In [48]:
import torch
import os
import numpy as np
import random
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import random
import pickle
import operator
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
random.seed(3407)
np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed(3407)
torch.cuda.manual_seed_all(3407)

In [49]:
from network import Encoder


### Data Processing 

In [50]:
classes = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
                        'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax',
                        'Consolidation', 'Edema', 'Emphysema', 'Fibrosis',
                        'Pleural']
def imageToLabel(image_name):
    labels = image_name.split('_')[2].split('|')
    #print(labels)
    label = [1 if dis in labels else 0 for dis in classes]
    return np.array(label)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
def imageDetails(image_path):
    image_name = image_path.split('/')[-1]
    label_string = image_name.split('_')[2].split('|')
    #print(label_string)
    label_string = [la[:3] for la in label_string]
    image_label = imageToLabel(image_name)
    image = np.load(image_path)
    '''plt.imshow(image, 'gray')
    plt.title(label_string, fontsize=21)
    plt.axis('off')
    plt.show()'''
    if len(image.shape)!= 2:
        image = np.mean(image,axis=-1)

    image = Image.fromarray(image)
    tensor_image = transform(image).unsqueeze(0).cuda()
    return image_label, tensor_image

## Hamming Distance Learing During Training (Table 4)

In [51]:
x_1_path = './Dataset/train/trainChest_X-ray_Atelectasis|Consolidation|Effusion|Infiltration_46292.npy'
x_2_path= './Dataset/train/trainChest_X-ray_Atelectasis|Consolidation|Infiltration|Pneumonia_55941.npy'
x_3_path= './Dataset/train/trainChest_X-ray_Consolidation|Effusion|Infiltration_17699.npy'
x_4_path= './Dataset/train/trainChest_X-ray_Effusion|Infiltration_99837.npy'
x_5_path= './Dataset/train/trainChest_X-ray_Atelectasis|Consolidation|Effusion_5787.npy'

In [52]:
y_1, x_1 = imageDetails(x_1_path)
y_2, x_2 = imageDetails(x_2_path)
y_3, x_3 = imageDetails(x_3_path)
y_4, x_4 = imageDetails(x_4_path)
y_5, x_5 = imageDetails(x_5_path)

In [53]:
def model_load(hash_code):
    encoder = Encoder(len(classes),hash_code)
    if torch.cuda.is_available():
        encoder.cuda()
    model_name = f'JaccHash_{hash_code}.pkl'
    dataStorePath = './Datastore/Models/'
    model_path = os.path.join(dataStorePath,model_name)
    #print(model_path)
    encoder.load_state_dict(torch.load(model_path))
    return encoder

In [54]:
def HDCompare(image, image1, labels, labels1, hash_code):
    encoder = model_load(hash_code)
    _, h = encoder(image)
    _, h1 = encoder(image1)
    distance_file_mname = f'distances_{hash_code}.pkl'
    with open(os.path.join('./Datastore/Distances/', distance_file_mname), 'rb') as file:
        distG = pickle.load(file)
    cos = F.cosine_similarity(h, h1, dim=1, eps=1e-6)
    cos_distH = F.relu((1-cos)*hash_code/2)
    #print(distH)

    sum_label = labels +labels1
    #print(labels[0])
    #print(labels1[0])
    #print(sum_label)

    union_label = (sum_label >= 1).sum().tolist()
    intersection_label = (sum_label >= 2).sum().tolist()
    #print(union_label)
    #print(intersection_label)
    g_distH = distG[union_label][intersection_label]

    g_distH = torch.tensor(g_distH)
    print('Groundtruth HD:', g_distH.item())
    print('Predicted HD:', cos_distH.item())
    print('-----------')
    return cos_distH.item(), g_distH.item()


##### Please specify the hash code length for HD comparision

In [55]:

hash_code_length =16
print('comparison between ground truth HD  and predicted HD:')
predicted_hd_12, g_distH_12 = HDCompare(x_1, x_2, y_1, y_2, hash_code=hash_code_length)
predicted_hd_13, g_distH_13 = HDCompare(x_1, x_3, y_1, y_3, hash_code=hash_code_length)
predicted_hd_14, g_distH_14 = HDCompare(x_1, x_4, y_1, y_4, hash_code=hash_code_length)
predicted_hd_15, g_distH_15 = HDCompare(x_1, x_5, y_1, y_5, hash_code=hash_code_length)
Predicted_distance_list = [predicted_hd_12, predicted_hd_13, predicted_hd_14, predicted_hd_15]
G_distH_list = [g_distH_12, g_distH_13, g_distH_14, g_distH_15]
print('Mean and Std, of Groundtruth HD:', [np.mean(G_distH_list), np.std(G_distH_list)])
print('Mean and Std, of Predicted HD:', [np.mean(Predicted_distance_list), np.std(Predicted_distance_list)])


comparison between ground truth HD  and predicted HD:
Groundtruth HD: 6
Predicted HD: 5.220604419708252
-----------
Groundtruth HD: 4
Predicted HD: 3.7795209884643555
-----------
Groundtruth HD: 8
Predicted HD: 7.707073211669922
-----------
Groundtruth HD: 4
Predicted HD: 4.11698579788208
-----------
Mean and Std, of Groundtruth HD: [5.5, 1.6583123951777]
Mean and Std, of Predicted HD: [5.206046104431152, 1.5391860506020445]


## Top Retrieval images (Figure 5)

In [56]:
from metrics import nDCG, aCG, mAPw

In [57]:
encoder = model_load(hash_code=48) #results in paper are produced using 48-bit gash code length

In [58]:
galleryfolderpath ='./Dataset/gallery'
queryfolderpath = './Dataset/query'
gallery_files = os.listdir(galleryfolderpath)
query_files = os.listdir(queryfolderpath)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [59]:
gallery = {}
print("\n\n Building Gallery .... \n")
with torch.no_grad():
    # Process each gallery image
    for img in gallery_files:
        image_path = os.path.join(galleryfolderpath, img)

        # Load and transform the image
        image = np.load(image_path)
        # transfer to one channel
        if len(image.shape)!= 2:
            image = np.mean(image,axis=-1)

        image = Image.fromarray(image)
        tensor_image = transform(image).unsqueeze(0).cuda()

        # Pass the tensor through the medianet model
        x_e, h = encoder(tensor_image)

        # Store the result in the gallery dictionary
        gallery[img] = torch.sign(h) #binary code

        # Clean up
        del tensor_image
    print("\n Building Complete. \n")

    count = 0




 Building Gallery .... 


 Building Complete. 



In [60]:
q_name = 'Chest_X-ray_Atelectasis|Emphysema|Infiltration_27411.npy'
query_image_path = os.path.join(queryfolderpath, q_name)
# Load and transform the image
query_image = np.load(query_image_path)
# transfer to one channel
if len(query_image.shape)!= 2:
    query_image = np.mean(query_image,axis=-1)
query_image = Image.fromarray(query_image)
query_tensor_image = transform(query_image).unsqueeze(0).cuda()

# Pass the tensor through the medianet model
_, h_q = encoder(query_tensor_image)

dist = {}
for key, h1 in gallery.items():
    #h1norm = torch.div(h1, torch.norm(h1, p=2))
    #h2norm = torch.div(torch.sign(h_q), torch.norm(h_q, p=2))
    #dist[key] = torch.pow(torch.norm(h1norm - h2norm, p=2), 2) * hash_code / 4
    #print(h1)
    #print(torch.sign(h_q))
    cos = F.cosine_similarity(h1, torch.sign(h_q), dim=1, eps=1e-6)
    dist[key] = F.relu((1-cos)*h_q.shape[1]/2)

sorted_pool = sorted(dist.items(), key=operator.itemgetter(1))[0:10]
sorted_pool

[('Chest_X-ray_Atelectasis_73317.npy',
  tensor([0.], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis_108334.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis|Infiltration_35321.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis_111942.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis_9120.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis|Effusion|Infiltration_91774.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis|Pleural_Thickening_84179.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis|Effusion|Fibrosis_23127.npy',
  tensor([2.0000], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Chest_X-ray_Atelectasis|Pneumonia_72747.npy',
  tensor([21.], device='cuda:0', grad_fn=<ReluBackward0>)),
 ('Che